### Parameters:
&emsp;$x$: number of assignees\
&emsp;$t$: estimated project workload for a single dev (hours)\
&emsp;$r_1$: intra-group productivity coefficient\
&emsp;$r_2$: inter-group productivity coefficient\
&emsp;$a$: Number of groups assigned to the project\
&emsp;$c$: average capability

### Functions:
&emsp;$I(x)$: Individual productivity

$$I(x)=r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c$$


&emsp;$G(x)$: Group productivity

$$G(x)=x \cdot I(x)=x \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c$$


&emsp;$T(x)$: Function of the estimated project time with x  assignees (hours)

$$T(x)=\frac{t}{G(x)}=\frac{t}{x \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}$$


&emsp;$C(x)$: Function of the estimated project cost with $x$ assignees (in dev hours)

$$C(x)=\frac{t}{I(x)}=\frac{t}{r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}$$


&emsp;$M(x)$: marginal benefit of increase assignees

$$M(x)=-\frac{dT}{dx}-\frac{dC}{dx}=\frac{t \cdot (x \cdot ln(r1) \cdot (1+x)+a)}{a \cdot x^2 \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}, x \geq a$$


&emsp;$R(x)$: Threshold r1, minimum r1 required to have a non-negative margin with given number of groups and number of assignees

$$R(x)=e^{-\frac{a}{x \cdot (1+x)}}$$



In [4]:
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
import math


def f(x, t, r1, r2, a, c):
    try:
        x = int(x)
        t = float(t)
        r1 = float(r1)
        r2 = float(r2)
        a = int(a)
        c = float(c)
    except:
        print(
            "please enter numbers,decimal will round down to nearest integer when necessary"
        )
        out = None
        df1 = None
    else:
        if x <= 0 or t <= 0 or r1 <= 0 or r2 <= 0 or a <= 0 or c <= 0:
            print("please enter non zeros")
            return None
        if r1 > 1 or r1 < 0.5:
            print("warning: r1 should between 0.5 and 1")
        if r2 > 1 or r2 < 0.5:
            print("warning: r2 should between 0.5 and 1")
        table1 = {
            "x": [],
            "a": [],
            "Individual productivity": [],
            "overall productivity": [],
            "Estimated time (100 dev hours)": [],
            "Estimated cost (100 dev hours)": [],
            "project time average rate of change": [],
            "cost average rate of change": [],
            "margin (100hours)": [],
        }
        table2 = {"x": [], "-dT/dx - dC/dx (100 hours)": []}
        table3 = {"x": [], "r1": []}
        initial = True
        last_tp = None
        last_cp = None
        for x_i in range(max(1, x - 10), x + 10):
            table1["x"].append(x_i)
            a_i = min(x_i, a)
            table1["a"].append(a_i)
            i_p = r1 ** (x_i / a_i - 1) * r2 ** (a_i - 1) * c
            table1["Individual productivity"].append(i_p)
            g_p = x_i * i_p
            table1["overall productivity"].append(g_p)
            t_p = t / g_p / 100
            table1["Estimated time (100 dev hours)"].append(t_p)
            c_p = t / i_p / 100
            table1["Estimated cost (100 dev hours)"].append(c_p)
            if initial:
                table1["project time average rate of change"].append(None)
                table1["cost average rate of change"].append(None)
                table1["margin (100hours)"].append(None)
                initial = False
            else:
                t_rc = t_p - last_tp
                table1["project time average rate of change"].append(t_rc)
                c_rc = c_p - last_cp
                table1["cost average rate of change"].append(c_rc)
                table1["margin (100hours)"].append(-t_rc - c_rc)
            last_cp = c_p
            last_tp = t_p
            if x_i >= a:
                table2["x"].append(x_i)
                table2["-dT/dx - dC/dx (100 hours)"].append(
                    t
                    * (x_i * math.log(r1) * (1 + x_i) + a)
                    / (a * x_i**2 * r1 ** (x / a - 1) * r2 ** (a - 1) * c)
                    / 100
                )
                table3["x"].append(x_i)
                table3["r1"].append(math.exp(-a / (x_i * (1 + x_i))))
        df1 = pd.DataFrame(table1)
        df2 = pd.DataFrame(table2)
        df3 = pd.DataFrame(table3)
        display(df1)
        plt.plot(
            df1["x"], df1["Individual productivity"], label="Individual productivity"
        )
        plt.plot(df1["x"], df1["overall productivity"], label="overall productivity")
        plt.plot(
            df1["x"],
            df1["Estimated time (100 dev hours)"],
            label="Estimated time (100 dev hours)",
        )
        plt.legend()
        plt.title("Graph1: Productivity")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        plt.plot(
            df1["x"],
            df1["Estimated time (100 dev hours)"],
            label="Estimated time (100 dev hours)",
        )
        plt.plot(
            df1["x"],
            df1["Estimated cost (100 dev hours)"],
            label="Estimated cost (100 dev hours)",
        )
        plt.legend()
        plt.title("Graph2: estimated workload(100 hours)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        plt.plot(
            df1["x"],
            df1["project time average rate of change"],
            label="project time average rate of change",
        )
        plt.plot(
            df1["x"],
            df1["cost average rate of change"],
            label="cost average rate of change",
        )
        plt.plot(df1["x"], df1["margin (100hours)"], label="margin (100hours)")
        plt.legend()
        plt.title("Graph 3: Marginal benefit (average rate of change)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        display(df2)
        plt.plot(
            df2["x"],
            df2["-dT/dx - dC/dx (100 hours)"],
            label="project time average rate of change",
        )
        plt.legend()
        plt.title("Graph4: Marginal benefit, -dT/dx - dC/dx (100 hours)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        display(df3)
        plt.plot(df3["x"], df3["r1"], label="r1")
        plt.legend()
        plt.title("Graph5: threshold r1")
        plt.grid(True, which="both")
        plt.show()
    return None


# a=interact(f,x='number of assignee',t='project workload',r1='intra-group productivity coefficient',r2='inter-group productivity coefficient',a='number of groups',c='average capability')
a = interact(f, x="1", t="1600", r1="0.8", r2="0.8", a="2", c="1")


interactive(children=(Text(value='1', description='x'), Text(value='1600', description='t'), Text(value='0.8',…